In [1]:
!pip install selenium


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd

페이지 별 링크 수집

In [44]:
def collect_by_page(page):


    driver = webdriver.Chrome()
    driver.get(f'https://kin.naver.com/search/list.naver?query=%EC%A0%95%EB%B6%8024&section=qna&page={page}')

    time.sleep(2)

    # tbody 안의 모든 tr 요소 가져오기
    fqa_table = driver.find_element(By.ID, 's_content')
    fqa_table_body = fqa_table.find_element(By.CLASS_NAME, 'basic1')
    fqa_links = fqa_table_body.find_elements(By.XPATH, './li/dl/dt/a')

    # href 속성 값을 저장할 리스트 생성
    href_list = []

    # 모든 a 태그에서 href 속성 추출
    for link in fqa_links:
        href = link.get_attribute('href')
        href_list.append(href)

    # # 이제 각 href 링크로 접근
    # for href in href_list:
    #     print("들어와")
    #     print(href)
    #     driver.get(href)

    #     time.sleep(2)  # 페이지 로딩 대기

    #     driver.back()
    #     time.sleep(2)  # 이전 페이지로 돌아간 후 대기

    driver.quit()
    
    return href_list


In [45]:
total_links = []
for i in range(30):
    links = collect_by_page(i+1)
    total_links += links
    

링크 csv파일로 저장

In [46]:
df = pd.DataFrame(total_links, columns=['Link'])

# CSV 파일로 저장
df.to_csv('naver_minwon/links.csv', index=False)

이제 각 href 링크로 접근하여 질문 및 답변 수집   
{"질문": , "답변": []} 형식으로 수집

In [52]:
driver = webdriver.Chrome()

fna_list = []

try:

    for link in total_links:
        driver.get(link)
        time.sleep(2)  # 페이지 로딩 대기

        full_content = driver.find_element(By.ID, 'content')

        question = full_content.find_element(By.CSS_SELECTOR, '.contentArea._contentWrap')
        question_title = question.find_element(By.CLASS_NAME, 'endTitleSection').text.replace('질문','').strip()
        question_content = question.find_element(By.CLASS_NAME, 'questionDetail').text
        print(question_title, question_content)

        answer_list = []
        try:
            answers = full_content.find_elements(By.CSS_SELECTOR, '._contentBox.contentBox')
            for answer in answers:
                answer_content = answer.find_element(By.CSS_SELECTOR, '.answerDetail._endContents._endContentsText').text
                answer_list.append(answer_content)
            
        except:
            print(f"{link}: answer없음")

        fna_list.append({'질문':question_title + " " + question_content, '답변':answer_list})

        driver.back()
        time.sleep(2)  # 이전 페이지로 돌아간 후 대기
except:
    print(question_title, question_content)
    print(answer_list)
    print(fna_list)

민원 내용 .csv파일로 저장

In [51]:
df = pd.DataFrame(fna_list)

# CSV 파일로 저장
df.to_csv('naver_minwon/qa_pairs.csv', index=False, encoding='utf-8-sig')

In [ ]:
s